In [1]:
import requests
import zipfile
import pandas as pd
import io
from sqlalchemy import create_engine

In [2]:
# generate list of file names to download and clean
years = ['2013','2014','2015','2016','2017', '2018','2019', '2020']
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
save_directory = '../../../Data_Sets/citibike/'
db_name = '201306-202101-citibike-tripdata.sqlite'
engine = create_engine(f'sqlite:///{save_directory}{db_name}')
file_names = [f'{year}{month}-citibike-tripdata' for year in years for month in months]
file_names.append('202101-citibike-tripdata')

In [3]:
def extract_clean(save_path, file_list):
    header_format = {
        'tripduration':'trip_duration',
        'bikeid':'ride_id',
        'bike_id':'ride_id',
        'starttime':'started_at',
        'stoptime':'ended_at',
        'start station name':'start_station_name',
        'start station id':'start_station_id',
        'end station name':'end_station_name',
        'end station id':'end_station_id',
        'start station latitude':'start_lat',
        'start station longitude':'start_lng',
        'end station latitude':'end_lat',
        'end station longitude':'end_lng',
        'usertype':'member_casual',
        'birth year':'Birth Year',
        'gender':'Gender',
        'trip duration':'trip_duration',
        'Bike ID':'ride_id',
        'Start Time':'started_at',
        'Stop Time':'ended_at',
        'Start Station Name':'start_station_name',
        'Start Station ID':'start_station_id',
        'End Station Name':'end_station_name',
        'End Station ID':'end_station_id',
        'Start Station Latitude':'start_lat',
        'Start Station Longitude':'start_lng',
        'End Station Latitude':'end_lat',
        'End Station Longitude':'end_lng',
        'User Type':'member_casual',
        'Birth Year':'Birth Year',
        'Trip Duration':'trip_duration'
    }
    for file in file_list:
        url1 = 'https://s3.amazonaws.com/tripdata/' + file + '.zip'
        url2 = 'https://s3.amazonaws.com/tripdata/' + file + '.csv.zip'
        
        #attempt to locate the specified file
        response = requests.get(url1)
        if response.status_code != 200:
            response = requests.get(url2)
            if response.status_code != 200:
                print(f'{file} is unavailable')
                continue

        # rename the column headers, standardize entry formats, and insert the file contents into a sqlite database
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_contents:
            file_list = zip_contents.namelist()
            with zip_contents.open(file_list[0]) as tempfile:
                bike_df = pd.read_csv(tempfile)
                bike_df = bike_df.rename(columns=header_format)
                bike_df['started_at'] = (pd.to_datetime(bike_df['started_at'])
                                         .dt.strftime('%Y-%m-%d %H:%M:%S'))
                bike_df['ended_at'] = (pd.to_datetime(bike_df['ended_at'])
                                       .dt.strftime('%Y-%m-%d %H:%M:%S'))
                bike_df['member_casual'] = bike_df['member_casual'].replace({
                    'Subscriber':'member',
                    'Customer':'casual'
                })
                bike_df.sort_values(by=['started_at'],ignore_index=True)    
                bike_df.to_sql('bikedata', con = engine, if_exists='append',index=False)
            tempfile.close()
        zip_contents.close()
        response.close()
        del url1, url2, response 
        del file_list, tempfile, bike_df, zip_contents
        print(f'Successfully extracted and cleaned {file}')

In [4]:
extract_clean(f'{save_directory}{db_name}', file_names)

201301-citibike-tripdata is unavailable
201302-citibike-tripdata is unavailable
201303-citibike-tripdata is unavailable
201304-citibike-tripdata is unavailable
201305-citibike-tripdata is unavailable
Successfully extracted and cleaned 201306-citibike-tripdata
Successfully extracted and cleaned 201307-citibike-tripdata
Successfully extracted and cleaned 201308-citibike-tripdata
Successfully extracted and cleaned 201309-citibike-tripdata
Successfully extracted and cleaned 201310-citibike-tripdata
Successfully extracted and cleaned 201311-citibike-tripdata
Successfully extracted and cleaned 201312-citibike-tripdata
Successfully extracted and cleaned 201401-citibike-tripdata
Successfully extracted and cleaned 201402-citibike-tripdata
Successfully extracted and cleaned 201403-citibike-tripdata
Successfully extracted and cleaned 201404-citibike-tripdata
Successfully extracted and cleaned 201405-citibike-tripdata
Successfully extracted and cleaned 201406-citibike-tripdata
Successfully extract

# Statistics on Bikes

In [4]:
# the WHERE statement here is used to remove rows where the bike_id is not tracked
bikeinfo_df = pd.read_sql("""
SELECT 
    ride_id AS "Bike ID",
    MIN(started_at) AS "First Ride",
    MAX(started_at) AS "Last Ride",
    SUM(ROUND((JULIANDAY(ended_at) - JULIANDAY(started_at)) * 86400)) AS "Total Ride Time (Seconds)",
    COUNT(started_at) AS "Total Rides"
FROM bikedata
WHERE strftime('%Y-%m', started_at) < '2021-02'
GROUP BY ride_id;
    """,con = engine)

In [5]:
bikeinfo_df['Life Span (Years)'] = pd.to_datetime(bikeinfo_df['Last Ride']).dt.year-pd.to_datetime(bikeinfo_df['First Ride']).dt.year

In [6]:
display(bikeinfo_df)
display(bikeinfo_df.info())

,Bike ID,First Ride,Last Ride,Total Ride Time (Seconds),Total Rides,Life Span (Years)
0,14529,2013-06-05 00:09:05,2020-11-09 08:59:36,8781904.0,8753,7
1,14530,2014-01-14 19:18:18,2020-12-11 20:06:11,7225321.0,7272,6
2,14531,2013-06-01 11:24:44,2020-11-09 15:42:37,7559708.0,7830,7
3,14532,2013-06-02 08:57:20,2019-05-26 08:03:36,7052137.0,6810,6
4,14533,2013-06-06 11:09:23,2020-12-01 16:59:28,7939666.0,8695,7
...,...,...,...,...,...,...
33102,50105,2020-12-12 14:15:19,2020-12-16 08:48:07,9040.0,12,0
33103,50106,2020-12-21 07:54:39,2020-12-24 17:17:11,5280.0,7,0
33104,50107,2020-12-18 10:27:38,2020-12-18 10:27:38,668.0,1,0
33105,50123,2021-01-18 12:07:01,2021-01-30 15:39:34,48260.0,61,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33107 entries, 0 to 33106
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Bike ID                    33107 non-null  int64  
 1   First Ride                 33107 non-null  object 
 2   Last Ride                  33107 non-null  object 
 3   Total Ride Time (Seconds)  33107 non-null  float64
 4   Total Rides                33107 non-null  int64  
 5   Life Span (Years)          33107 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 1.5+ MB


None

In [19]:
bikeinfo_df.to_csv(f'{save_directory}bike_lifespan.csv')

# Statistics on Stations

In [18]:
# retrieve statistics on the number of trips starting at each station
start_stationinfo = pd.read_sql("""
SELECT 
    start_station_name AS "Station Name",
    strftime('%Y-%m', started_at) AS Month,
    start_station_id AS "Station ID",
    start_lat AS "Station Latitude",
    start_lng AS "Station Longitude",
    COUNT(start_station_id) AS "Trips From",
    COUNT(DISTINCT end_station_name) AS "Stations Visited"
FROM bikedata
GROUP BY Month, "Station Name";
    """,con = engine)

In [22]:
start_stationinfo = start_stationinfo.dropna(axis=0)
display(start_stationinfo)
display(start_stationinfo.info())

,Station Name,Month,Station ID,Station Latitude,Station Longitude,Trips From,Stations Visited
0,1 Ave & E 16 St,2013-06,504.0,40.732219,-73.981656,2331,274
1,1 Ave & E 18 St,2013-06,2003.0,40.733812,-73.980544,1690,261
2,1 Ave & E 30 St,2013-06,536.0,40.741444,-73.975361,1692,248
3,1 Ave & E 44 St,2013-06,455.0,40.750020,-73.969053,1728,254
4,10 Ave & W 28 St,2013-06,489.0,40.750664,-74.001768,2094,251
...,...,...,...,...,...,...,...
57130,Woodward Ave & Harman St,2021-01,3888.0,40.707930,-73.910920,128,61
57131,Wyckoff Av & Jefferson St,2021-01,3763.0,40.707165,-73.923711,279,113
57132,Wyckoff Av & Stanhope St,2021-01,3780.0,40.703545,-73.917775,424,140
57133,Wyckoff St & Nevins St,2021-01,3911.0,40.683426,-73.984275,457,155


<class 'pandas.core.frame.DataFrame'>
Int64Index: 57125 entries, 0 to 57134
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Station Name       57125 non-null  object 
 1   Month              57125 non-null  object 
 2   Station ID         57125 non-null  float64
 3   Station Latitude   57125 non-null  float64
 4   Station Longitude  57125 non-null  float64
 5   Trips From         57125 non-null  int64  
 6   Stations Visited   57125 non-null  int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 3.5+ MB


None

In [36]:
# retrieve statistics on the number of trips starting at each station
end_stationinfo = pd.read_sql("""
SELECT 
    end_station_name AS "Station Name",
    strftime('%Y-%m', ended_at) AS Month,
    end_station_id AS "Station ID",
    end_lat AS "Station Latitude",
    end_lng AS "Station Longitude",
    COUNT(end_station_id) AS "Trips To",
    COUNT(DISTINCT start_station_name) AS "Stations Departed From"
FROM bikedata
GROUP BY Month, "Station Name";
    """,con = engine)

In [37]:
end_stationinfo = end_stationinfo.dropna(axis=0)
display(end_stationinfo)
display(end_stationinfo.info())

,Station Name,Month,Station ID,Station Latitude,Station Longitude,Trips To,Stations Departed From
1,1 Ave & E 16 St,2013-06,504.0,40.732219,-73.981656,2328,283
2,1 Ave & E 18 St,2013-06,2003.0,40.733812,-73.980544,1636,269
3,1 Ave & E 30 St,2013-06,536.0,40.741444,-73.975361,1588,251
4,1 Ave & E 44 St,2013-06,455.0,40.750020,-73.969053,1729,269
5,10 Ave & W 28 St,2013-06,489.0,40.750664,-74.001768,2046,253
...,...,...,...,...,...,...,...
57813,W 4 St & 7 Ave S,2021-02,380.0,40.734011,-74.002939,1,1
57814,W 95 St & Broadway,2021-02,3314.0,40.793770,-73.971888,1,1
57815,Washington Ave & Park Ave,2021-02,313.0,40.696102,-73.967510,1,1
57816,Water - Whitehall Plaza,2021-02,534.0,40.702551,-74.012723,1,1


<class 'pandas.core.frame.DataFrame'>
Int64Index: 57806 entries, 1 to 57817
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Station Name            57806 non-null  object 
 1   Month                   57806 non-null  object 
 2   Station ID              57806 non-null  float64
 3   Station Latitude        57806 non-null  float64
 4   Station Longitude       57806 non-null  float64
 5   Trips To                57806 non-null  int64  
 6   Stations Departed From  57806 non-null  int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 3.5+ MB


None

In [44]:
# Retrieve statistics on number of trips departing from each station
merged_stationinfo = (pd.merge(start_stationinfo,
                               end_stationinfo,
                               on=['Station Name','Month','Station ID',
                                   'Station Latitude','Station Longitude'],
                               how = 'outer'))

In [45]:
# Nans here refer to instances where a trip was not take to or from a station in a given month.
# Replacing with 0 for analysis
merged_stationinfo = merged_stationinfo.fillna(0)

In [46]:
display(merged_stationinfo)
display(merged_stationinfo.info())
display(merged_stationinfo.isna().any())

,Station Name,Month,Station ID,Station Latitude,Station Longitude,Trips From,Stations Visited,Trips To,Stations Departed From
0,1 Ave & E 16 St,2013-06,504.0,40.732219,-73.981656,2331.0,274.0,2328.0,283.0
1,1 Ave & E 18 St,2013-06,2003.0,40.733812,-73.980544,1690.0,261.0,1636.0,269.0
2,1 Ave & E 30 St,2013-06,536.0,40.741444,-73.975361,1692.0,248.0,1588.0,251.0
3,1 Ave & E 44 St,2013-06,455.0,40.750020,-73.969053,1728.0,254.0,1729.0,269.0
4,10 Ave & W 28 St,2013-06,489.0,40.750664,-74.001768,2094.0,251.0,2046.0,253.0
...,...,...,...,...,...,...,...,...,...
57832,W 4 St & 7 Ave S,2021-02,380.0,40.734011,-74.002939,NaN,NaN,1.0,1.0
57833,W 95 St & Broadway,2021-02,3314.0,40.793770,-73.971888,NaN,NaN,1.0,1.0
57834,Washington Ave & Park Ave,2021-02,313.0,40.696102,-73.967510,NaN,NaN,1.0,1.0
57835,Water - Whitehall Plaza,2021-02,534.0,40.702551,-74.012723,NaN,NaN,1.0,1.0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 57837 entries, 0 to 57836
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Station Name            57837 non-null  object 
 1   Month                   57837 non-null  object 
 2   Station ID              57837 non-null  float64
 3   Station Latitude        57837 non-null  float64
 4   Station Longitude       57837 non-null  float64
 5   Trips From              57125 non-null  float64
 6   Stations Visited        57125 non-null  float64
 7   Trips To                57806 non-null  float64
 8   Stations Departed From  57806 non-null  float64
dtypes: float64(7), object(2)
memory usage: 4.4+ MB


None

Station Name              False
Month                     False
Station ID                False
Station Latitude          False
Station Longitude         False
Trips From                 True
Stations Visited           True
Trips To                   True
Stations Departed From     True
dtype: bool

In [50]:
merged_stationinfo.to_csv(f'{save_directory}station_history.csv')